In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from tqdm import tqdm
import time

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def equation_plane(v0, v1, v2):
    point_mat = np.array([v0, v1, v2])
    abc = np.matmul(np.linalg.inv(point_mat), np.array([[1], [1], [1]]))
    p = np.concatenate([abc.T, np.array(-1).reshape(1, 1)], axis=1) / (np.sum(abc ** 2) ** 0.5)
    p = p.reshape(4)
    
    return p

In [3]:
# calculate K
def calculate_K(vertices, triangles):
    K_list = []

    for i in range(len(triangles)):
        t = triangles[i]
        v0, v1, v2 = vertices[t[0]], vertices[t[1]], vertices[t[2]]
        p = equation_plane(v0, v1, v2)
        K = np.outer(p, p)
        K_list.append(K)
                
    return K_list

In [4]:
# create dict {v_id: [t_id0, t_id1, ...]}
def create_v_tr(triangles):
    v_tr = {}

    for i in range(len(triangles)):
        t = triangles[i]
        for v in t:
            if v in v_tr:
                v_tr[v].append(i)
            else:
                v_tr[v] = [i]
                
    return v_tr

In [5]:
def calculate_Q(K_list, v_tr):
    v_Q = {}

    for v_id, tr_ids in v_tr.items():
        Q = sum([K_list[el] for el in tr_ids])
        v_Q[v_id] = Q
        
    return v_Q

In [6]:
def find_edges(triangles):
    edge_1 = triangles[:, 0:2]
    edge_2 = triangles[:, 1:]
    edge_3 = np.concatenate([triangles[:, :1], triangles[:, -1:]], axis=1)
    edges = np.concatenate([edge_1, edge_2, edge_3], axis=0)
    
    unique_edges_trans, unique_edges_locs = np.unique(edges[:, 0] * (10 ** 10) + edges[:, 1], return_index=True)
    edges = edges[unique_edges_locs, :]
    
    return edges

In [7]:
def select_valid_pairs(vertices, triangles, t=0.001):
    
    valid_pairs = np.array([])
    
    for i in tqdm(range(len(vertices))):
        v = vertices[i]
        distances = np.sum((vertices - v) ** 2, axis=1) ** 0.5
        valid = np.where(distances < t)[0]
        pairs = np.vstack((np.ones(len(valid)) * i, valid)).T
        
        if i == 0:
            valid_pairs = pairs
        else:
            valid_pairs = np.vstack((valid_pairs, pairs))

    valid_pairs = valid_pairs[valid_pairs[:, 0] != valid_pairs[:, 1]]
    valid_pairs = valid_pairs.astype(int)
    
    edges = find_edges(triangles)
    valid_pairs = np.vstack((valid_pairs, edges))
    
    valid_pairs = valid_pairs[valid_pairs[:, 0] != valid_pairs[:, 1]]

    unique_valid_pairs_trans, unique_valid_pairs_loc = np.unique(valid_pairs[:, 0] * (10 ** 10) + valid_pairs[:, 1], return_index=True)
    valid_pairs = valid_pairs[unique_valid_pairs_loc, :]
    
    return valid_pairs

In [8]:
def calculate_error(valid_pairs, v_Q, vertices):
    errors = []
    
    b = np.ones((vertices.shape[0], 1))
    vertices_4d = np.hstack((vertices, b))

    for i in range(len(valid_pairs)):
        pair = valid_pairs[i]
        v = (vertices_4d[pair[0]] - vertices_4d[pair[1]]) / 2
        
        try:
            Q1 = v_Q[pair[0]]
            Q2 = v_Q[pair[1]]
            Q = Q1 + Q2
        
            error = np.dot(np.dot(v, Q), v.T)    
        except KeyError:
            error = 10 ** 10
        
        errors.append(error)
        
    return errors

In [9]:
def create_queue(valid_pairs, errors):
    queue = np.hstack((valid_pairs, np.reshape(errors, (-1, 1))))
    # queue = queue[queue[:, 2] != 10 ** 10]
    queue = queue[np.argsort(queue[:, 2])]
    
    return queue

In [10]:
def update_queue(queue, v0, v1, v_Q, vertices):
    # drop invalid pairs from queue for v1
    queue = queue[~((queue[:, 0] == v1) | (queue[:, 1] == v1))]

    # update error value for v0
    value_pairs_to_update = queue[(queue[:, 0] == v0) | (queue[:, 1] == v0)][:, :2].astype(int)
    errors_to_update = np.array(calculate_error(value_pairs_to_update, v_Q, vertices))

    rows = np.argwhere((queue[:, 0] == v0) | (queue[:, 1] == v0))    
    queue[rows, 2] = np.reshape(errors_to_update, (-1, 1))
    
    # sort again
    queue = queue[np.argsort(queue[:, 2])]
    
    return queue

In [19]:
def remove_one_vertice(v0, v1, vertices, triangles, v_tr):
    vertices[v0] = (vertices[v0] + vertices[v1]) / 2

    # 2. change v2 to v1 in planes
    try:
        for tr in v_tr[v1]:
            triangles[tr][triangles[tr] == v1] = v0
    except KeyError:
        pass

    return vertices, triangles

In [11]:
def remove_vertices3(vertices, triangles, ratio, t):

    valid_pairs = select_valid_pairs(vertices, triangles, t=t)
    num_steps = int(len(triangles) * (1 - ratio) / 2)
    K_list = calculate_K(vertices, triangles)
    v_tr = create_v_tr(triangles)
    v_Q = calculate_Q(K_list, v_tr)
    errors = calculate_error(valid_pairs, v_Q, vertices)
    queue = create_queue(valid_pairs, errors)
    
    for i in tqdm(range(num_steps)):
        
        v0, v1, error = queue[0]
        v0, v1 = int(v0), int(v1)
        v_Q[v0] = v_Q[v0] + v_Q[v1]
        del v_Q[v1]
        
        vertices, triangles = remove_one_vertice(v0, v1, vertices, triangles, v_tr)
        valid_pairs = valid_pairs[~((valid_pairs[:, 0] == v1) | (valid_pairs[:, 1] == v1))]

        triangles = np.unique(triangles, axis=0)
        triangles = triangles[(triangles[:, 0] != triangles[:, 1]) & (triangles[:, 1] != triangles[:, 2]) & (triangles[:, 2] != triangles[:, 0])]

        v_tr = create_v_tr(triangles)
        queue = update_queue(queue, v0, v1, v_Q, vertices)
        
        if len(queue) == 0:
            break

    return vertices, triangles

In [17]:
def mesh_simplify(mesh, ratio, t):
    print(mesh)

    vertices = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)

    vertices_new, triangles_new = remove_vertices3(vertices, triangles, ratio=ratio, t=t)
    
    mesh_new = o3d.geometry.TriangleMesh()
    mesh_new.vertices = o3d.utility.Vector3dVector(vertices_new)
    mesh_new.triangles = o3d.utility.Vector3iVector(triangles_new)

    mesh_new.remove_unreferenced_vertices()
    print(mesh_new)
    
    return mesh_new

In [20]:
mesh = o3d.geometry.TriangleMesh.create_torus()
mesh_new = mesh_simplify(mesh, ratio=0.5, t=1)
o3d.io.write_triangle_mesh('torus_ratio0.5_t1.obj', mesh_new)

TriangleMesh with 600 points and 1200 triangles.


100%|██████████| 300/300 [00:01<00:00, 171.27it/s]

TriangleMesh with 300 points and 600 triangles.


True

In [37]:
mesh = o3d.geometry.TriangleMesh.create_torus()
mesh_new = mesh_simplify(mesh, ratio=0.1, t=1)
o3d.io.write_triangle_mesh('torus_ratio0.1_t1.obj', mesh_new)

TriangleMesh with 600 points and 1200 triangles.


100%|██████████| 540/540 [00:02<00:00, 249.97it/s]

TriangleMesh with 60 points and 120 triangles.


True

In [36]:
mesh = o3d.geometry.TriangleMesh.create_torus()
mesh_new = mesh_simplify(mesh, ratio=0.5, t=0.01)
o3d.io.write_triangle_mesh('torus_ratio0.5_t0.01.obj', mesh_new)

TriangleMesh with 600 points and 1200 triangles.


100%|██████████| 300/300 [00:00<00:00, 723.22it/s]

TriangleMesh with 300 points and 600 triangles.


True

In [26]:
mesh = o3d.geometry.TriangleMesh.create_torus()
mesh_new = mesh_simplify(mesh, ratio=0.05, t=1)
o3d.io.write_triangle_mesh('torus_ratio0.05_t1.obj', mesh_new)

TriangleMesh with 600 points and 1200 triangles.


100%|██████████| 570/570 [00:02<00:00, 254.92it/s]


TriangleMesh with 30 points and 60 triangles.


True

In [39]:
mesh_path = 'bunny.obj'
mesh = o3d.io.read_triangle_mesh(mesh_path,True)

mesh_new = mesh_simplify(mesh, ratio=0.05, t=0)
o3d.io.write_triangle_mesh('torus_ratio0.05_t1.obj', mesh_new)

TriangleMesh with 192031 points and 69451 triangles.


  1%|▏         | 2453/192031 [00:13<17:09, 184.07it/s]


KeyboardInterrupt: 